In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import random

In [2]:
img_list = sorted(glob.glob('./image2/*.jpg'))
mask_list = sorted(glob.glob('./mask2/*.jpg'))

print(len(img_list), len(mask_list))
print(img_list[0], mask_list[0])

5712 5712
./image2/001_000067.jpg ./mask2/001_000067.jpg


In [3]:
# Preprocessing
mask_cv = []
img_cv = []
for i in range(len(mask_list)):
    mask = cv2.imread(mask_list[i], 0)
    ret,mask = cv2.threshold(mask,127,255,cv2.THRESH_BINARY)
    mask_cv.append(mask)

# img 256x256
for i in range(len(img_list)):
    img = cv2.imread(img_list[i], 0)
    img_cv.append(img)
    

In [4]:
indices = np.where(mask_cv[5] != [0])
y, x = indices[0], indices[1]
coordinates = np.array((x,y))
print(sum(x)/len(x), sum(y)/len(y))


338.7404927691484 262.19469737546865


In [43]:
# Size Consider

def SemiCenter(img, mask, size):
    
    indices = np.where(mask != [0])
    y, x = indices[0], indices[1]
    a, b = (min(x), min(y)) # Lower Left Corner 
    W = (max(x) - min(x)) # Width  
    h = (max(y) - min(y)) # Height
    
    x = int(a+W/2) # Center x
    y = int(b+h/2) # Center y
    
    s_h = max(0, (size - h)/2) # Height Space
    s_w = max(0, (size - W)/2) # Height Width
    
    x_spare = min(s_w, W)/2 # x 축 여유
    y_spare = min(s_h, h)/2 # y 축 여유
    #print(x, y, x_spare, y_spare)
    x_new = random.randint(int(x-x_spare), int(x+x_spare))
    y_new = random.randint(int(y-y_spare), int(y+y_spare)) # New CoG
    
    if size <= 256:
        
        if size/2 < y_new < 511 - size/2: # 범위 안 벗어나면 y, x 업데이트. 벗어나면 그냥 무게중심으로 
            y = y_new
            
        if size/2 < x_new < 511 - size/2:
            x = x_new
    
    else:
        if a + size/2 >= 510:
            x = 510 - size/2
        elif a - size/2 <= 0:
            x = size/2 
        else:
            if size/2 < y_new < 512 - size/2: # 범위 안 벗어나면 y, x 업데이트. 벗어나면 그냥 무게중심으로 
                y = y_new
                
        if b + size/2 >= 510:
            y = 510 - size/2
        elif b - size/2 <= 0:
            y = size/2
        else:
            if size/2 < x_new < 512 - size/2:
                x = x_new
    
    #print(x_new, y_new, x, y)
    mask = mask[int(y-size/2):int(y+size/2), int(x-size/2):int(x+size/2)]
    img = img[int(y-size/2):int(y+size/2), int(x-size/2):int(x+size/2)]
        
    return img, mask

In [44]:
# Test
for i in range(len(img_list)):
    
    img_path = img_list[i][9:19]
    mask_path = mask_list[i][8:18]
    
    img, mask = SemiCenter(img_cv[i], mask_cv[i], 144)


In [49]:
# Saving RandomCrop

for i in range(len(img_list)):
    
    img_path = img_list[i][9:19]
    mask_path = mask_list[i][8:18]

    for j in range(3):
    
        img, mask = SemiCenter(img_cv[i], mask_cv[i], 144)
        cv2.imwrite('./Crop_Semirand/Image_Sr_144/' + img_path + '_' + str(j+1) + '.jpg', img)
        cv2.imwrite('./Crop_Semirand/Mask_Sr_144/' + mask_path + '_' + str(j+1) + '.jpg', mask)
        